# Fluxo de Carga Convencional Newtonimport numpy as np

## Imports

In [12]:
import numpy as np
import cmath
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import lu_factor, lu_solve
# import sympy as sy

## Newton Method Class

In [13]:
class newtonMethod():
    
    def __init__(self, bus_df, branch_df, tolerancia, limitador = 200, only_Y=False):
        
        # Bus columns    = 'Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'
        # Branch columns = 'From', 'To', 'r', 'x', 'Bsh', 'Tap'
        
        self.bus_df    = bus_df
        self.branch_df = branch_df
        
        self.bus_len = len(self.bus_df)
        self.bra_len = len(self.branch_df)
        self.limitador = limitador
        
        self.iteracao   = []
        self.tolerancia = tolerancia
        self.titulo = []
        
        self.n_mismatches = 0
        self.P_mis, self.Q_mis = 0, 0
        self.slack_pos = 0
        self.PV_pos, self.PQ_pos = [], []
        
        for i in range(self.bus_len):
            
            if self.bus_df.loc[i+1,['Type']][0] == 'VO':
                self.slack_pos = i
                
            
            if self.bus_df.loc[i+1,['Type']][0] == 'PQ':
                
                self.n_mismatches += 2
                self.P_mis += 1
                self.Q_mis += 1
                self.PQ_pos.append(i)
                
            if self.bus_df.loc[i+1,['Type']][0] == 'PV':
                
                self.n_mismatches += 1
                self.P_mis += 1
                self.PV_pos.append(i)
                
    # ==========================================================================================
    # ======================== MATRIX GENERATOR ================================================
    # ==========================================================================================      
        
    def createMatrix(self):
        
        self.Z     = np.zeros((self.bus_len, self.bus_len), dtype=np.complex128)
        self.b     = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.a     = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.theta = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.E     = np.zeros((self.bus_len), dtype=float)
        self.O     = np.zeros((self.bus_len), dtype=float)
        self.Pesp  = np.zeros((self.bus_len), dtype=float)
        self.Qesp  = np.zeros((self.bus_len), dtype=float)
        
        for i in range(self.bra_len):
            
            k = self.branch_df.loc[i,['From']].to_numpy(dtype=int)[0]-1
            m = self.branch_df.loc[i,['To']].to_numpy(dtype=int)[0]-1

            r = self.branch_df.loc[i,['r']].to_numpy(dtype=float)[0]
            x = self.branch_df.loc[i,['x']].to_numpy(dtype=float)[0]
            
            z = complex(r, x)

            self.Z[k, m] = z
            self.Z[m, k] = z
            
            self.b[k, m] = self.branch_df.loc[i,['Bsh']].to_numpy(dtype=float)[0]/2
            self.b[m, k] = self.branch_df.loc[i,['Bsh']].to_numpy(dtype=float)[0]/2
            
            self.a[k, m] = self.branch_df.loc[i,['Tap']].to_numpy(dtype=float)[0]
            self.a[m, k] = 1 / (self.branch_df.loc[i,['Tap']].to_numpy(dtype=float)[0])
            
            self.theta[k, m] = self.bus_df.loc[k+1, ['Theta']].to_numpy(dtype=float)[0] - self.bus_df.loc[m+1, ['Theta']].to_numpy(dtype=float)[0]
            self.theta[m, k] = self.bus_df.loc[m+1, ['Theta']].to_numpy(dtype=float)[0] - self.bus_df.loc[k+1, ['Theta']].to_numpy(dtype=float)[0]
            
        
            
        for i in range(self.bus_len):
            
            self.b[i,i]   = self.bus_df.loc[i+1,['Bsh']].to_numpy(dtype=float)[0]
            self.E[i]     = self.bus_df.loc[i+1,['V']].to_numpy(dtype=float)[0]
            self.O[i]     = self.bus_df.loc[i+1,['Theta']].to_numpy(dtype=float)[0]
            self.Pesp[i]  = self.bus_df.loc[i+1,['Pg']].to_numpy(dtype=float)[0] - self.bus_df.loc[i+1,['Pc']].to_numpy(dtype=float)[0]
            self.Qesp[i]  = self.bus_df.loc[i+1,['Qg']].to_numpy(dtype=float)[0] - self.bus_df.loc[i+1,['Qc']].to_numpy(dtype=float)[0]
        
        self.Y  = np.zeros((self.bus_len, self.bus_len), dtype=np.complex128)
        self.yk = np.zeros((self.bus_len, self.bus_len), dtype=np.complex128)
        
        for i in range(self.bus_len):
            for j in range(self.bus_len):

                if self.Z[i,j] != 0 + 0j:
                    
                    self.yk[i,j] = 1 / self.Z[i,j]
                    self.Y[i,j] = -self.a[i,j]*(1 / self.Z[i,j])

                else:
                    self.yk[i,j] = 0 + 0j
                    self.Y[i,j]  = 0 + 0j


        for i in range(self.bus_len):

            soma = 0

            for j in range(self.bus_len):

                if j != i:

                    soma += complex(0, self.b[i,j]) + (self.a[i,j]**2) * self.yk[i,j]

            soma = soma + complex(0, self.b[i,i])

            self.Y[i,i] = soma
    
    # ==========================================================================================
    # ======================== PV AND PQ =======================================================
    # ==========================================================================================    
                
    def allPower(self):
        
        self.delta = np.zeros((self.bus_len*2), dtype=float)
        
        self.Q = np.zeros((self.bus_len), dtype=float)
        self.P = np.zeros((self.bus_len), dtype=float)
        
        for i in range(self.bus_len):

            for j in range(self.bus_len):
                
                   
                self.P[i] += self.E[j]*(np.real(self.Y[i,j])*np.cos(self.theta[i, j]) + np.imag(self.Y[i,j])*np.sin(self.theta[i, j]))

                self.Q[i] += self.E[j]*(np.real(self.Y[i,j])*np.sin(self.theta[i, j]) - np.imag(self.Y[i,j])*np.cos(self.theta[i, j]))


            self.P[i] = self.E[i]*self.P[i]
            self.Q[i] = self.E[i]*self.Q[i]

        for i in range(self.bus_len):
            
            self.delta[i]                = self.Pesp[i] - self.P[i]
            self.delta[i + self.bus_len] = self.Qesp[i] - self.Q[i]
            
    # ==========================================================================================
    # ======================== JACOBI MATRIX ===================================================
    # ==========================================================================================    
    
    def subMatrix(self):
        
        self.H = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.N = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.M = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.L = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.J = np.zeros((2*self.bus_len, 2*self.bus_len), dtype=float)
        
        # ======== H
        
        for k in range(self.bus_len):
            for m in range(self.bus_len):
                
                if m > k:
                    
                    self.H[k, m] =   self.E[k]*self.E[m]*(np.real(self.Y[k, m])*np.sin(self.theta[k, m]) - np.imag(self.Y[k, m])*np.cos(self.theta[k, m]))
                    self.H[m, k] = - self.E[k]*self.E[m]*(np.real(self.Y[k, m])*np.sin(self.theta[k, m]) + np.imag(self.Y[k, m])*np.cos(self.theta[k, m]))
                    
                if m == k:
                    soma = 0
                    for i in range(self.bus_len):
                        soma += self.E[i]*(np.real(self.Y[k, i])*np.sin(self.theta[k, i]) - np.imag(self.Y[k, i])*np.cos(self.theta[k, i]))
                        
                    
                    self.H[m, k] = - np.imag(self.Y[k, k])*(self.E[k]**2) - soma*self.E[k]
                    
        # ======== N           
                    
        for k in range(self.bus_len):
            for m in range(self.bus_len):
                
                if m > k:
                    
                    self.N[k, m] = self.E[k]*(np.real(self.Y[k, m])*np.cos(self.theta[k, m]) + np.imag(self.Y[k, m])*np.sin(self.theta[k, m]))
                    self.N[m, k] = self.E[m]*(np.real(self.Y[k, m])*np.cos(self.theta[k, m]) - np.imag(self.Y[k, m])*np.sin(self.theta[k, m]))
                    
                if m == k:
                    soma = 0
                    for i in range(self.bus_len):
                        soma += self.E[i]*(np.real(self.Y[k, i])*np.cos(self.theta[k, i]) + np.imag(self.Y[k, i])*np.sin(self.theta[k, i]))
                        
                    
                    self.N[m, k] = np.real(self.Y[k, k])*self.E[k] + soma
                    
        # ======== M
        
        for k in range(self.bus_len):
            for m in range(self.bus_len):
                
                if m > k:
                    
                    self.M[k, m] = - self.E[k]*self.E[m]*(np.real(self.Y[k, m])*np.cos(self.theta[k, m]) + np.imag(self.Y[k, m])*np.sin(self.theta[k, m]))
                    self.M[m, k] = - self.E[k]*self.E[m]*(np.real(self.Y[k, m])*np.cos(self.theta[k, m]) - np.imag(self.Y[k, m])*np.sin(self.theta[k, m]))
                    
                if m == k:
                    soma = 0
                    for i in range(self.bus_len):
                        soma += self.E[i]*(np.real(self.Y[k, i])*np.cos(self.theta[k, i]) + np.imag(self.Y[k, i])*np.sin(self.theta[k, i]))
                        
                    
                    self.M[m, k] = - np.real(self.Y[k, k])*(self.E[k]**2) + soma
                    
        # ======== l
        
        for k in range(self.bus_len):
            for m in range(self.bus_len):
                
                if m > k:
                    
                    self.L[k, m] =   self.E[k]*(np.real(self.Y[k, m])*np.sin(self.theta[k, m]) - np.imag(self.Y[k, m])*np.cos(self.theta[k, m]))
                    self.L[m, k] = - self.E[m]*(np.real(self.Y[k, m])*np.sin(self.theta[k, m]) + np.imag(self.Y[k, m])*np.cos(self.theta[k, m]))
                    
                if m == k:
                    soma = 0
                    for i in range(self.bus_len):
                        soma += self.E[i]*(np.real(self.Y[k, i])*np.sin(self.theta[k, i]) - np.imag(self.Y[k, i])*np.cos(self.theta[k, i]))
                        
                    
                    self.L[m, k] = - np.imag(self.Y[k, k])*self.E[k] + soma
                    
        # ======== J
        
        j1     = np.concatenate((self.H, self.N), axis=1)
        j2     = np.concatenate((self.M, self.L), axis=1)
        self.J = np.concatenate((j1, j2), axis=0)
        
        self.J[self.slack_pos, self.slack_pos] = np.Inf
        self.J[self.slack_pos + self.bus_len, self.slack_pos + self.bus_len] = np.Inf
        
        for pos in self.PV_pos:
            self.J[pos + self.bus_len, pos + self.bus_len] = np.Inf
        
    # ==========================================================================================
    # ======================== MODEL FUNCTION ==================================================
    # ==========================================================================================   
    
    def model(self):
        
        
        for i in range(self.limitador):
            
            inter = []
        
            self.createMatrix()
            self.allPower()
            self.subMatrix()

            self.J_inv     = np.linalg.inv(self.J)
            self.delta_ans = np.dot(self.J_inv, self.delta)
            
            

            if np.max(np.abs(self.delta_ans)) < self.tolerancia:
                
                for i in self.PQ_pos:

                    self.O[i] += self.delta_ans[i]
                    self.E[i] += self.delta_ans[i + self.bus_len]

                for i in self.PV_pos:

                    self.O[i] += self.delta_ans[i]

                self.bus_df['Theta'] = self.O
                self.bus_df['V']     = self.E
                                
                if np.argmax(np.abs(self.delta_ans)) < self.bus_len:
                    inter.append("P" +str(np.argmax(np.abs(self.delta_ans)) + 1))
                else:
                    inter.append("Q" +str(np.argmax(np.abs(self.delta_ans)) - self.bus_len + 1))
                
                inter.append(np.max(np.abs(self.delta_ans)))
                inter.append(np.max(np.abs(self.delta_ans)) < self.tolerancia)
                    
                self.iteracao.append(inter)
                    
                break
            
            for i in self.PQ_pos:

                self.O[i] += self.delta_ans[i]
                self.E[i] += self.delta_ans[i + self.bus_len]
                
            for i in self.PV_pos:

                self.O[i] += self.delta_ans[i]

            self.bus_df['Theta'] = self.O
            self.bus_df['V']     = self.E
             
            if np.argmax(np.abs(self.delta_ans)) < self.bus_len:
                inter.append("P" +str(np.argmax(np.abs(self.delta_ans)) + 1))
            else:
                inter.append("Q" +str(np.argmax(np.abs(self.delta_ans)) - self.bus_len + 1))
                
            inter.append(np.max(np.abs(self.delta_ans)))
            inter.append(np.max(np.abs(self.delta_ans)) < self.tolerancia)
            
            self.iteracao.append(inter)
            
        self.titulo.append('Variable')
        self.titulo.append('Max(|Mis|)')
        self.titulo.append('<' + str(self.tolerancia))
            
        self.allPower()
        
        self.tabela = pd.DataFrame(self.iteracao, columns=self.titulo)
        
        for i in range(self.bus_len):
            
            if i == self.slack_pos:
                if self.P[i] >= 0:
                    self.bus_df.at[i+1, 'Pg'] = self.P[i]
                else:
                    self.bus_df.at[i+1, 'Pc'] = np.abs(self.P[i])
                    
                if self.Q[i] >= 0:
                    self.bus_df.at[i+1, 'Qg'] = self.Q[i]
                else:
                    self.bus_df.at[i+1, 'Qc'] = np.abs(self.Q[i])
                    
            if i in self.PV_pos:
                
                if self.Q[i] >= 0:
                    self.bus_df.at[i+1, 'Qg'] = self.Q[i]
                else:
                    self.bus_df.at[i+1, 'Qc'] = np.abs(self.Q[i])        

## Programa

In [14]:
BusData = np.loadtxt('ieee5bus.txt',skiprows=2)
BranchData = np.loadtxt('ieee5branch.txt',skiprows=2)

In [16]:
Pb = 100                # Potência base para conversão para PU
tolerancia = 0.0001     # Tolerancia
iteracoes = 200         # Numero máximo de iterações

In [17]:
for i in range(len(BusData)):
    BusData[i, 4:7] = BusData[i, 4:7] / Pb


In [18]:
i = [i+1 for i in range(len(BusData))]

df_bus = pd.DataFrame(BusData[:, 0],columns=['Bus'], index = i)

tipo = ["PQ", "NaN", "PV", "VO"]
tipo_b = []
for i in range(len(BusData)):
  tipo_b.append(tipo[int(BusData[i, 1])])

df_bus['Type'] = tipo_b

df_bus['Pg']    = BusData[:, 6]
df_bus['Pc']    = BusData[:, 4]
df_bus['Qg']    = BusData[:, 7]
df_bus['Qc']    = BusData[:, 5]
df_bus['V']     = BusData[:, 2]
df_bus['Theta'] = BusData[:, 3]
df_bus['Bsh']   = BusData[:, 8]

df_bus

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,0.0,0.00,-0.0,0.00,1.0,0.0,0.0
2,2.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
3,3.0,PQ,0.0,0.25,0.0,0.15,1.0,0.0,0.0
4,4.0,PQ,0.0,0.40,0.0,0.20,1.0,0.0,0.0
5,5.0,PQ,0.0,0.50,0.0,0.20,1.0,0.0,0.0


In [19]:
df_branch = pd.DataFrame(BranchData, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap', 'Phi'])
df_branch

,From,To,r,x,Bsh,Tap,Phi
0,1.0,2.0,0.05,0.11,0.02,1.0,0.0
1,1.0,3.0,0.05,0.11,0.02,1.0,0.0
2,1.0,5.0,0.03,0.08,0.02,1.0,0.0
3,2.0,3.0,0.04,0.09,0.02,1.0,0.0
4,2.0,5.0,0.04,0.09,0.02,1.0,0.0
5,3.0,4.0,0.06,0.13,0.03,1.0,0.0
6,4.0,5.0,0.04,0.09,0.02,1.0,0.0


In [20]:
m = newtonMethod(df_bus, df_branch, tolerancia, iteracoes)
m.model()

In [21]:
m.tabela

,Variable,Max(|Mis|),<0.0001
0,Q4,0.061554,False
1,Q4,0.005093,False
2,P4,0.000034,True


In [22]:
m.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.597904,0.00,0.726009,0.00,1.000000,0.000000,0.0
2,2.0,PQ,0.000000,0.40,0.000000,0.20,0.955306,-0.041972,0.0
3,3.0,PQ,0.000000,0.25,0.000000,0.15,0.954818,-0.041256,0.0
4,4.0,PQ,0.000000,0.40,0.000000,0.20,0.933334,-0.063701,0.0
5,5.0,PQ,0.000000,0.50,0.000000,0.20,0.953394,-0.046931,0.0


In [24]:
bus_bonito = m.bus_df.copy()

In [27]:
precisao = 3

for i in range(len(bus_bonito)):

    bus_bonito["Theta"][i+1] = round(np.degrees(m.bus_df["Theta"][i+1]), precisao)

    bus_bonito["Pg"][i+1] = round(m.bus_df["Pg"][i+1], precisao)
    bus_bonito["Pc"][i+1] = round(m.bus_df["Pc"][i+1], precisao)
    bus_bonito["Qg"][i+1] = round(m.bus_df["Qg"][i+1], precisao)
    bus_bonito["Qc"][i+1] = round(m.bus_df["Qc"][i+1], precisao)

    bus_bonito["V"][i+1] = round(m.bus_df["V"][i+1], precisao)   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A va

In [28]:
bus_bonito

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1.0,VO,1.598,0.00,0.726,0.00,1.000,0.000,0.0
2,2.0,PQ,0.000,0.40,0.000,0.20,0.955,-2.405,0.0
3,3.0,PQ,0.000,0.25,0.000,0.15,0.955,-2.364,0.0
4,4.0,PQ,0.000,0.40,0.000,0.20,0.933,-3.650,0.0
5,5.0,PQ,0.000,0.50,0.000,0.20,0.953,-2.689,0.0
